In [3]:
import os
import glob
import pandas as pd
import numpy as np 
import pingouin as pg 
import matplotlib.pyplot as plt
import warnings
import seaborn as sns

from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor

warnings.filterwarnings("ignore")

In [4]:
curr_dir = os.getcwd()
avg_dir = curr_dir +"/AVG" 

rating_data = pd.read_csv( avg_dir + "/Dynamic_Rating.csv", index_col=0).values
rating_data.shape

(52, 16)

In [5]:
data_list = os.listdir(avg_dir)
data_list = sorted([ data for data in data_list if not data.startswith(".") and data != 'Dynamic_Rating.csv' and not data.endswith(".npy")])

avg_data = { str(x).split(sep=".")[0] : pd.read_csv(f"{avg_dir}/{x}", index_col=0).values for x in data_list}

avg_arr = [ ]

for ind, x in avg_data.items() : 
    #avg_arr.append(x.mean(axis=0)) # 16개 영상별로 평균을 내는 경우
    avg_arr.append(x.mean(axis=1)) # 52명 데이터를 평균내는 경우
    #avg_arr.append(x.flatten())

avg_arr = np.array(avg_arr).T

for key in avg_data.keys() : 
    print (key)
    print(avg_data[key].shape)
    print("-"*20)

print(avg_arr.shape)

Blinking
(52, 16)
--------------------
Fixation_Duration
(52, 16)
--------------------
Fixation_Num
(52, 16)
--------------------
Saccade_Duration
(52, 16)
--------------------
left_iris
(52, 16)
--------------------
left_pupil
(52, 16)
--------------------
right_iris
(52, 16)
--------------------
right_pupil
(52, 16)
--------------------
(52, 8)


In [6]:
rating_data.shape

(52, 16)

In [7]:
# Block별 데이터 

data_list = os.listdir(avg_dir)
data_list = sorted([ data for data in data_list if not data.startswith(".") and data != 'Dynamic_Rating.csv' and not data.endswith(".npy")])

#block_rating = rating_data.reshape(len(rating_data),-1,4).mean(axis=0).T # 16개 영상별로 평균을 내는 경우
block_rating = rating_data.reshape(len(rating_data),-1,4).mean(axis=1).T # 52명 데이터를 평균내는 경우

avg_data = { str(x).split(sep=".")[0] : pd.read_csv(f"{avg_dir}/{x}", index_col=0).values for x in data_list}

block_arr = [ ]
for key, value in avg_data.items() : 
    #block_arr.append(value.reshape(len(value),-1,4).mean(axis=0)) # 16개 영상별로 평균을 내는 경우 -> (video num, Block num, Eye factors)
    block_arr.append(value.reshape(len(value),-1,4).mean(axis=1)) # 52명 데이터를 평균내는 경우

block_arr = np.array(block_arr).T
print(block_arr.shape) 
print("-"*20,"\n")

(4, 52, 8)
-------------------- 



In [8]:
# SSQ 데이터 
items = ['Bleary' , 'Dry_Eyed' , 'Eyestrain', 'Gritty', 'Eye_Ache', 'Sting', \
        'Heavy_Eyes', 'Hazy', 'Warm_Eyes', 'Flickering', 'Watery_Eyes', 'Feeling_heavy_in_the_head', 'Feel_heavy', \
        'Difficulty_concentrating', 'Dizzy', 'Stiff_shoulder', 'Stiff_neck', 'Sleepy', 'Vomiiting', 'Vertigo', 'Nausea', \
        'Difficulty_focussing', 'Double_vision', 'Near_vision_difficulty', \
        'Far_vision_difficulty', 'Pain_in_the_temple', 'Pain_in_the_middle_of_the_head', 'Pain_in_the_back_of_the_head' ]

ssq_arr = np.load(avg_dir+"/ssq.npy")

ssq_arr = ssq_arr[..., 1:]

ssq_arr.shape

(28, 52, 4)

In [9]:
# Dynamic ratign DF

#arr = np.concatenate([avg_arr, rating_data.mean(axis=0).reshape(-1,1)], axis=1) # 16개 영상별로 평균을 내는 경우
arr = np.concatenate([avg_arr, rating_data.mean(axis=1).reshape(-1,1)], axis=1) # 52명 데이터를 평균내는 경우
#arr = np.concatenate([avg_arr, rating_data.flatten().reshape(-1,1)], axis=1)

print( "arr shape : ", arr.shape)

names = list(avg_data.keys())
names.append('DynamicRating')

#df = pd.DataFrame(arr, columns= names )# 16개 영상별로 평균을 내는 경우
df = pd.DataFrame(np.concatenate([avg_arr, rating_data.mean(axis=1).reshape(-1,1)], axis=1 ), columns= names )# 52명 데이터를 평균내는 경우
#df = pd.DataFrame(arr, columns=names)

print("Dataframe shape :",df.shape)



arr shape :  (52, 9)
Dataframe shape : (52, 9)


In [10]:
# Dynamic ratign DF Min_Max scaling 

def min_max_scaler( series) :
    min = series.min()
    max = series.max()
    x = (series - min ) / ( max - min ) 
    return x 

#df = df.apply( min_max_scaler)

In [11]:
VIF             = pd.DataFrame()
VIF['VIF']      = [variance_inflation_factor(df.iloc[:,:-1].values, i) for i in range(df.iloc[:,:-1].shape[1])]
VIF['Variable'] = df.iloc[:,:-1].columns

VIF

,VIF,Variable
0,13.280531,Blinking
1,94.530617,Fixation_Duration
2,173.752270,Fixation_Num
3,7.707274,Saccade_Duration
4,158.025092,left_iris
5,771.283009,left_pupil
6,200.045989,right_iris
7,930.124524,right_pupil


In [12]:
df_vif = df[['Blinking','Fixation_Duration','Fixation_Num','Saccade_Duration']]
VIF             = pd.DataFrame()
VIF['VIF']      = [variance_inflation_factor(df_vif.values, i) for i in range(df_vif.shape[1])]
VIF['Variable'] = df_vif.columns

VIF

,VIF,Variable
0,7.254987,Blinking
1,12.996583,Fixation_Duration
2,26.738414,Fixation_Num
3,6.022749,Saccade_Duration


In [17]:
# "DynamicRating ~ Blinking + Fixation_Duration  + Fixation_Num + Saccade_Duration  + left_iris + right_iris + left_pupil +right_pupil"

m = ols("DynamicRating ~ Blinking + Fixation_Duration  + Fixation_Num + Saccade_Duration  + left_iris + right_iris + left_pupil +right_pupil", data=df).fit()
print(f"R vlaue : {np.sqrt(m.rsquared):.4f} || ANOVA p-value : {m.f_pvalue:.2f}")
print(m.summary())

R vlaue : 0.4920 || ANOVA p-value : 0.12
                            OLS Regression Results                            
Dep. Variable:          DynamicRating   R-squared:                       0.242
Model:                            OLS   Adj. R-squared:                  0.101
Method:                 Least Squares   F-statistic:                     1.717
Date:                Wed, 15 Nov 2023   Prob (F-statistic):              0.122
Time:                        14:25:33   Log-Likelihood:                -54.722
No. Observations:                  52   AIC:                             127.4
Df Residuals:                      43   BIC:                             145.0
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

In [38]:
m = ols("DynamicRating ~ Blinking ", data=df).fit()
 
print(f"R vlaue : {np.sqrt(m.rsquared):.4f} || ANOVA p-value : {m.f_pvalue:.2f}")

print(m.summary())

R vlaue : 0.3143 || ANOVA p-value : 0.02
                            OLS Regression Results                            
Dep. Variable:          DynamicRating   R-squared:                       0.099
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     5.479
Date:                Wed, 15 Nov 2023   Prob (F-statistic):             0.0233
Time:                        01:56:38   Log-Likelihood:                -59.225
No. Observations:                  52   AIC:                             122.5
Df Residuals:                      50   BIC:                             126.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  

In [39]:
# Block별 Dynamic rating LR
Blocks = ['HMHB', 'HMLB', "LMHB", "LMLB"]
for ind, value in enumerate(block_arr):
    block_df = pd.DataFrame(np.concatenate([value, block_rating[ind].reshape(-1,1)], axis=1 ), columns=names)
    #block_df.apply(min_max_scaler)
    #block_df.to_csv(f"./DR-{Blocks[ind]}.csv")
    #m = ols("DynamicRating ~ Blinking + Fixation_Duration  + Fixation_Num + Saccade_Duration  + left_iris + right_iris + left_pupil +right_pupil", data=block_df).fit()
    m = ols("DynamicRating ~ Blinking", data=block_df).fit()
    print(Blocks[ind])
    #print(f"R2 vlaue : {m.rsquared:.4f} || ANOVA p-value : {m.f_pvalue:.2f}")
    print(m.summary())
    print("="*100)
    print( "\n")


HMHB
                            OLS Regression Results                            
Dep. Variable:          DynamicRating   R-squared:                       0.053
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     2.818
Date:                Wed, 15 Nov 2023   Prob (F-statistic):             0.0994
Time:                        01:56:38   Log-Likelihood:                -60.123
No. Observations:                  52   AIC:                             124.2
Df Residuals:                      50   BIC:                             128.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.5005      0.284     12.332    

In [43]:
# SSQ_LR 

for item_ind, item in enumerate(items) : 

    names = list(avg_data.keys())
    names.append(item)

    #block_df = pd.DataFrame(np.concatenate([block_arr.mean(axis=1), ssq_arr[item_ind].mean(axis=0).reshape(-1,1)], axis=1 ), columns=names)
    block_df = pd.DataFrame(np.concatenate([avg_arr, ssq_arr[item_ind].mean(axis=1).reshape(-1,1)], axis=1 ), columns=names)
    #block_df.to_csv(f"./{items[item_ind]}.csv")

    # block_df.apply(min_max_scaler)
    m = ols(f"{item} ~ Blinking + Fixation_Duration  + Fixation_Num + Saccade_Duration  + left_iris + right_iris + left_pupil +right_pupil", data=block_df).fit()
    #m = ols(f"{item} ~ Blinking", data=block_df).fit()


    if  m.f_pvalue < 0.05  : # np.sqrt(m.rsquared) >= 0.4 and
        print(f"[{items[item_ind]}]")
        #print(f"R vlaue : {np.sqrt(m.rsquared):.4f}")
        #print(f"R2 vlaue : {m.rsquared:.4f}")
        #print(f"ANOVA p-value : {m.f_pvalue:.3f}")
        print(m.summary())
        print("="*100)
        print( "\n")



[Dry_Eyed]
                            OLS Regression Results                            
Dep. Variable:               Dry_Eyed   R-squared:                       0.334
Model:                            OLS   Adj. R-squared:                  0.210
Method:                 Least Squares   F-statistic:                     2.691
Date:                Wed, 15 Nov 2023   Prob (F-statistic):             0.0171
Time:                        02:19:08   Log-Likelihood:                -80.611
No. Observations:                  52   AIC:                             179.2
Df Residuals:                      43   BIC:                             196.8
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           -22.826

In [18]:
# SSQ_LR 

for item_ind, item in enumerate(items) : 

    names = list(avg_data.keys())
    names.append(item)

    #block_df = pd.DataFrame(np.concatenate([block_arr.mean(axis=1), ssq_arr[item_ind].mean(axis=0).reshape(-1,1)], axis=1 ), columns=names)
    block_df = pd.DataFrame(np.concatenate([avg_arr, ssq_arr[item_ind].mean(axis=1).reshape(-1,1)], axis=1 ), columns=names)
    #block_df.to_csv(f"./{items[item_ind]}.csv")

    # block_df.apply(min_max_scaler)
    #m = ols(f"{item} ~ Blinking + Fixation_Duration  + Fixation_Num + Saccade_Duration  + left_iris + right_iris + left_pupil +right_pupil", data=block_df).fit()
    m = ols(f"{item} ~ Blinking", data=block_df).fit()


    if  m.f_pvalue < 0.05  : # np.sqrt(m.rsquared) >= 0.4 and
        print(f"[{items[item_ind]}]")
        #print(f"R vlaue : {np.sqrt(m.rsquared):.4f}")
        #print(f"R2 vlaue : {m.rsquared:.4f}")
        #print(f"ANOVA p-value : {m.f_pvalue:.3f}")
        print(m.summary())
        print("="*100)
        print( "\n")


[Bleary]
                            OLS Regression Results                            
Dep. Variable:                 Bleary   R-squared:                       0.092
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     5.042
Date:                Wed, 15 Nov 2023   Prob (F-statistic):             0.0292
Time:                        14:29:45   Log-Likelihood:                -82.534
No. Observations:                  52   AIC:                             169.1
Df Residuals:                      50   BIC:                             173.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.9772      0.445      4.440

In [41]:
# Blokc별 SSQ LR

Blocks = ['HMHB', 'HMLB', "LMHB", "LMLB"]

# items = ['Bleary' , 'Dry_Eyed' , 'Eyestrain', 'Gritty', 'Eye_Ache', 'Sting', \
#         'Heavy_Eyes', 'Hazy', 'Warm_Eyes', 'Flickering', 'Watery_Eyes', 'Feeling_heavy_in_the_head', 'Feel_heavy', \
#         'Difficulty_concentrating', 'Dizzy', 'Stiff_shoulder', 'Stiff_neck', 'Sleepy', 'Vomiiting', 'Vertigo', 'Nausea', \
#         'Difficulty_focussing', 'Double_vision', 'Near_vision_difficulty', \
#         'Far_vision_difficulty', 'Pain_in_the_temple', 'Pain_in_the_middle_of_the_head', 'Pain_in_the_back_of_the_head' ]

#items = ['Dry_Eyed', 'Eyestrain', 'Eye_Ache', 'Heavy_Eyes', 'Flickering']

ssq_lr_dict = { }

for item_ind, item in enumerate(items) : 

    names = list(avg_data.keys())
    names.append(item)
    item_r = []

    for ind, value in enumerate(block_arr):        
        block_df = pd.DataFrame(np.concatenate([value, ssq_arr[item_ind][:,ind].reshape(-1,1)], axis=1 ), columns=names)
        #block_df.to_csv(f"./{items[item_ind]} - {Blocks[ind]}.csv")
        # block_df.apply(min_max_scaler)
        m = ols(f"{item} ~ Blinking + Fixation_Duration  + Fixation_Num + Saccade_Duration  + left_iris + right_iris + left_pupil +right_pupil", data=block_df).fit()
        print(f"{items[item_ind]} - {Blocks[ind]}")
        print(f"R vlaue : {np.sqrt(m.rsquared):.4f}")
        print(f"R2 vlaue : {m.rsquared:.4f}")
        print(f"ANOVA p-value : {m.f_pvalue:.3f}")
        #print(m.summary())
        print("="*100)
        print( "\n")

        item_r.append(np.sqrt(m.rsquared).round(3))

    ssq_lr_dict[item] = item_r


Bleary - HMHB
R vlaue : 0.2991
R2 vlaue : 0.0895
ANOVA p-value : 0.829


Bleary - HMLB
R vlaue : 0.3990
R2 vlaue : 0.1592
ANOVA p-value : 0.438


Bleary - LMHB
R vlaue : 0.3153
R2 vlaue : 0.0994
ANOVA p-value : 0.778


Bleary - LMLB
R vlaue : 0.4083
R2 vlaue : 0.1667
ANOVA p-value : 0.398


Dry_Eyed - HMHB
R vlaue : 0.5648
R2 vlaue : 0.3190
ANOVA p-value : 0.024


Dry_Eyed - HMLB
R vlaue : 0.4945
R2 vlaue : 0.2446
ANOVA p-value : 0.117


Dry_Eyed - LMHB
R vlaue : 0.4854
R2 vlaue : 0.2356
ANOVA p-value : 0.137


Dry_Eyed - LMLB
R vlaue : 0.4992
R2 vlaue : 0.2492
ANOVA p-value : 0.107


Eyestrain - HMHB
R vlaue : 0.4662
R2 vlaue : 0.2174
ANOVA p-value : 0.188


Eyestrain - HMLB
R vlaue : 0.5137
R2 vlaue : 0.2639
ANOVA p-value : 0.080


Eyestrain - LMHB
R vlaue : 0.4355
R2 vlaue : 0.1896
ANOVA p-value : 0.291


Eyestrain - LMLB
R vlaue : 0.6226
R2 vlaue : 0.3876
ANOVA p-value : 0.004


Gritty - HMHB
R vlaue : 0.5229
R2 vlaue : 0.2734
ANOVA p-value : 0.066


Gritty - HMLB
R vlaue : 0.4907
